In [23]:
import keras

Using TensorFlow backend.


In [24]:
from __future__ import absolute_import
from __future__ import print_function
import os
import itertools
import numpy as np
np.random.seed(1337)  # for reproducibility

#from kerosene.datasets import svhn2
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.models import load_model

import sys
import tarfile
import pickle
import matplotlib.pyplot as plt
import numpy as np, h5py 
import pandas as pd

In [4]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    X_train = save['train_dataset']
    y_train = save['train_labels']
    X_val = save['valid_dataset']
    y_val = save['valid_labels']
    X_test = save['test_dataset']
    y_test = save['test_labels']
    del save  
    print('Training data shape:', X_train.shape)
    print('Training label shape:',y_train.shape)
    print('Validation data shape:', X_val.shape)
    print('Validation label shape:', y_val.shape)
    print('Test data shape:', X_test.shape)
    print('Test label shape:', y_test.shape)

Training data shape: (230070, 32, 32, 1)
Training label shape: (230070, 6)
Validation data shape: (5684, 32, 32, 1)
Validation label shape: (5684, 6)
Test data shape: (13068, 32, 32, 1)
Test label shape: (13068, 6)


In [5]:
batch_size = 128
nb_classes = 11
nb_epoch = 12

X_train = np.concatenate([X_train, X_val])
y_train = np.concatenate([y_train, y_val])

In [6]:
print("{1} train samples, {2} channel{0}, {3}x{4}".format("" if X_train.shape[1] == 1 else "s", *X_train.shape))
print("{1}  test samples, {2} channel{0}, {3}x{4}".format("" if X_test.shape[1] == 1 else "s", *X_test.shape))
_, img_channels, img_rows, img_cols = X_train.shape

235754 train samples, 32 channels, 32x1
13068  test samples, 32 channels, 32x1


In [7]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [10]:
Y_test[0]

array([ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.])

In [ ]:
keras_svhn_model  = 'my_model.h5'
#model = load_model(keras_svhn_model)
#model.save(keras_svhn_model)

In [11]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

/home/ubuntu/anaconda2/envs/p/lib/python2.7/site-packages/keras/models.py:647: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 235754 samples, validate on 13068 samples
Epoch 1/12
235754/235754 [==============================] - 101s - loss: 7.9475 - val_loss: 5.7659
Epoch 2/12
235754/235754 [==============================] - 96s - loss: 6.7586 - val_loss: 5.3824
Epoch 3/12
235754/235754 [==============================] - 96s - loss: 6.4733 - val_loss: 5.2456
Epoch 4/12
235754/235754 [==============================] - 96s - loss: 6.3667 - val_loss: 5.2007
Epoch 5/12
235754/235754 [==============================] - 96s - loss: 6.3072 - val_loss: 5.1750
Epoch 6/12
235754/235754 [==============================] - 96s - loss: 6.2711 - val_loss: 5.1478
Epoch 7/12
235754/235754 [==============================] - 96s - loss: 6.2446 - val_loss: 5.1201
Epoch 8/12
235754/235754 [==============================] - 96s - loss: 6.2248 - val_loss: 5.1216
Epoch 9/12
235754/235754 [==============================] - 96s - loss: 6.2102 - val_loss: 5.0990
Epoch 10/12
235754/235754 [==============================] - 96s -

/home/ubuntu/anaconda2/envs/p/lib/python2.7/site-packages/keras/models.py:689: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


IndexError: invalid index to scalar variable.

In [13]:
score

5.0813532193793698

In [25]:
keras_svhn_model  = 'my_model.h5'
model = load_model(keras_svhn_model)
#model.save(keras_svhn_model)

In [17]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [22]:
#import urllib, cStringIO
#import matplotlib.image as mpimg
import numpy as np
import PIL
from PIL import Image

#from skimage import io
from six.moves.urllib.request import urlretrieve
import urllib

URL = r'https://www.timpson.co.uk/media/catalog/product/cache/2/image/330x330/9df78eab33525d08d6e5fb8d27136e95/5/9/599.jpg'
file_img = urlretrieve(URL,'a')
#file_img = io.imread(file_img[0])

#imsize = np.ndarray([1,2])
im = Image.open('a')
im = im.resize((32,32), PIL.Image.ANTIALIAS)#.convert('L')
im = np.asarray(im)
im = rgb2gray(im)
im = np.expand_dims(im, axis=2)
mean = np.mean(im, dtype='float32')
std = np.std(im, dtype='float32', ddof=1)
if std < 1e-4: std = 1.
im = (im - mean) / std
im.shape
#gray = rgb2gray(im) 


(32, 32, 1)

In [28]:
dataset = np.ndarray([1,32,32,1], dtype='float32')
dataset[0,:,:,:] = im[:,:,:]

In [29]:
res = model.predict(dataset, batch_size=1, verbose=0)

In [31]:
len(res)

1

In [34]:
np.argmax(res, 2)

ValueError: axis(=2) out of bounds